In [16]:
import pyodbc
import logging
from os import listdir
from os.path import isfile, join,splitext
import pandas as pd
import xlwings as xw
from secret import secrets as secrets
cx = pyodbc.connect("DSN=gf32;UID={};PWD={}".format(
    secrets.dbusr, secrets.dbpw))

def getbatchesfromdb():
    query= '''
   SELECT distinct 
       [UDF_BATCH_NO],
	   TRANSACTIONDATE
	   
     
        FROM [MAS_GFC].[dbo].[AP_INVOICEHISTORYHEADER]
        where convert(varchar(8),TRANSACTIONDATE,112) between '20190901' and '20191001'
        order by TRANSACTIONDATE desc
    
    '''
    cursor = cx.cursor()
    
    
    cursor.execute(query)
    rows = cursor.fetchall()
#     data=[]
#     for row in rows:
#         data.append(row)
    return rows
def getbatchinfo(batchno):
    query= '''
   SELECT  
       [UDF_BATCH_NO],
	   TRANSACTIONDATE,
	   VENDORNO,
	   INVOICENO,
	   HEADERSEQNO
     
        FROM [MAS_GFC].[dbo].[AP_INVOICEHISTORYHEADER]
        where UDF_BATCH_NO =?
        order by TRANSACTIONDATE desc
    
    '''
    cursor = cx.cursor()
    
    
    cursor.execute(query,batchno)
    rows = cursor.fetchone()
#     data=[]
#     for row in rows:
#         data.append(row)
    return rows
    

In [9]:
import os
print("Generating batch file list...")
batchfiles = [f for f in listdir('L:\\APBatches') if isfile(join('L:\\APBatches',f)) and f.endswith('.pdf')]
print("Generating POBatches list...")
pofiles = [f for f in listdir('L:\\POBatches') if isfile(join('L:\\POBatches',f)) and f.endswith('.pdf')]

filenames =[]
pofilenames=[]
for file in batchfiles:
    filenames.append(os.path.splitext(file)[0])
for file in pofiles:
    pofilenames.append(os.path.splitext(file)[0])
db=[]
rows = getbatchesfromdb()
for row in rows:
    
    db.append(str(row.UDF_BATCH_NO))

Generating batch file list...
Generating POBatches list...


In [12]:
missing=set(db)-set(filenames)
missing-=set(pofilenames)
print(len(missing))
print(missing)

69
{'05171', '05201', '85683', '85493', '05059', '85575', '05285', '85700', '85685', '05079', '05223', '05336', '05126', '05172', '85612', '05024', '05282', '05193', '05137', '19374', '05100', '85444', '05209', '85652', '05167', '85639', '85631', '05273', '05170', '05101', '85470', '85716', '05294', '85533', '05181', '85474', '85647', '05113', '85684', '05309', '05333', '05208', '85693', '05230', '05284', '05176', '05065', '85488', '85468', '05115', '05212', '85640', '05112', '85611', '85491', '85590', '85656', '85673', '05221', '05322', '85546', '05144', '05186', '05341', '85672', '85618', '85705', '05194', '05139'}


In [28]:
data={
    "batchno":"",
    "transactiondate":"",
    "vendorno":"",
    "invoiceno":"",
    "headerseqno":""
    
}
extra=[]
for rec in missing:
    batch=getbatchinfo(rec)
    data['batchno']=batch.UDF_BATCH_NO
    data['vendorno']=batch.VENDORNO
    data['invoiceno']=batch.INVOICENO
    data['headerseqno']=batch.HEADERSEQNO
    data['transactiondate']=batch.TRANSACTIONDATE
#     print(data)
    extra.append(data.copy())
# print(extra)

{'batchno': '05171', 'transactiondate': '20190913', 'vendorno': 'LINKPLU', 'invoiceno': '200-3183696', 'headerseqno': '000001'}
{'batchno': '05201', 'transactiondate': '20190917', 'vendorno': 'A1RESOU', 'invoiceno': '109847', 'headerseqno': '000001'}
{'batchno': '85683', 'transactiondate': '20190927', 'vendorno': 'CASH', 'invoiceno': 'NO CHARGE 4', 'headerseqno': '000000'}
{'batchno': '85493', 'transactiondate': '20190911', 'vendorno': 'CHRISCH', 'invoiceno': 'ST 2520', 'headerseqno': '000000'}
{'batchno': '05059', 'transactiondate': '20190903', 'vendorno': 'WELLSCC', 'invoiceno': 'CONC090219', 'headerseqno': '000000'}
{'batchno': '85575', 'transactiondate': '20190919', 'vendorno': 'CASH', 'invoiceno': '0913201900', 'headerseqno': '000000'}
{'batchno': '05285', 'transactiondate': '20190925', 'vendorno': 'WELLSCC', 'invoiceno': 'UNCI032219', 'headerseqno': '000001'}
{'batchno': '85700', 'transactiondate': '20190927', 'vendorno': 'ANIMASH', 'invoiceno': 'BN09190000', 'headerseqno': '0000

In [29]:
df = pd.DataFrame(extra)

In [30]:
wb= xw.Book()
sheet= wb.sheets['Sheet1']
sheet.range('A1').value=df